In [281]:
import numpy as np

In [105]:
print("Create a file called params.txt next to script file")
print("This file should contain 5 rows.Like so:")
print("50 -> Population Size")
print("0.7 -> Mutaion Probability [0-1]")
print("0.8 -> Crossover Probabilty [0-1]")
print("1 -> Crossover Method [1: Single-point, 2: Double-point, 3:uniform]")
print("20 -> Max iteration")
while input("Enter 1 when the file is ready: ") != "1":
    continue

Create a file called params.txt next to script file
This file should contain 5 rows.Like so:
50 -> Population Size
0.7 -> Mutaion Probability [0-1]
0.4 -> Crossover Probabilty [0-1]
1 -> Crossover Method [1: Single-point, 2: Double-point, 3:uniform]
20 -> Max iteration
Enter 1 when the file is ready: 1


In [280]:
class Gen:
    def __init__(self, value):
        self.value = str(value)
        
    def __repr__(self):
        return f"{self.value}"
    
    
class Chrom:
    def __init__(self, values):
        self.genes = np.array([Gen(value) for value in values])
        self.fitness = 0
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def __repr__(self):
        return f"{self.genes} => {self.fitness}"
        
        
class GA:
    def __init__(self, ps, mp, cp, cm, mi):
        self.population_size = int(ps)
        self.mutaion_probability = float(mp) 
        self.crossover_probabilty = float(cp) 
        self.crossover_method = int(cm)
        self.max_iter = int(mi)
        self.population = self.generatePopulation()
        self.f = lambda x: x**2 + 2 * x 
        self.maximum = list("11110")
        
   
    def generatePopulation(self): 
        pop = []
        for i in range(self.population_size):
            values = np.base_repr(np.random.randint(low=0, high=2, size=1)[0], 2).zfill(5)
            chrom = Chrom(values)
            pop.append(chrom)
        return pop
    
    def get_population(self):
        return self.population

    
    def fit(self, population):
        for i, chrom in enumerate(population):
            fit = [t==gen.value and 1 or 0 for t,gen in zip(self.maximum, chrom.genes)]
            fit = sum([3**(4-i)*x for i, x in enumerate(fit)])
            chrom.fitness = fit
            
    def print_population(self):
        for g in self.get_population():
            print(g)
    
    def select_parents(self, n_parents=4):
        return np.random.choice(self.get_population(), n_parents)
    
    def single_point_crossover(self, chroms):
        a, b = parents
        idx = np.random.randint(1, len(a.genes)-1, 1)[0]
        c1, c2 = np.hstack((a.genes[:idx],  b.genes[idx:])), np.hstack((b.genes[:idx] , a.genes[idx:]))
        c1 = Chrom(c1)
        c2 = Chrom(c2)
        return [c1, c2] 
    
    def double_point_crossover(self, parents):
        a, b = parents
        i1, i2 = np.random.randint(1, len(a.genes)-1, 2)
        i1, i2 = min(i1, i2), max(i1, i2)
        c1, c2 = np.hstack((a.genes[:i1] , b.genes[i1:i2], a.genes[i2:])), np.hstack(( b.genes[:i1] , a.genes[i1:i2] , b.genes[i2:]))
        c1 = Chrom(c1)
        c2 = Chrom(c2)
        return [c1, c2]  
    
            
    def uniform_crossover(self, parents):
        a, b = parents
        cross_candidates = np.random.randint(0, 2, len(a.genes))
        c1, c2 = a.genes, b.genes
        for i in range(len(cross_candidates)):
            if cross_candidates[i]:
                c1[i], c2[i] = c2[i], c1[i]
        c1 = Chrom(c1)
        c2 = Chrom(c2)
        return [c1, c2]
    
    def _mutate(self, selected_parents):
        a,b = selected_parents
        i1, i2 = np.random.randint(0, len(a.genes), 2)
        c1, c2 = a.genes, b.genes
        c1[i1] = c1[i1].value == "1" and Gen("0") or Gen("1") 
        c1[i2] = c1[i2].value == "1" and Gen("0") or Gen("1") 
        c2[i1] = c2[i1].value == "1" and Gen("0") or Gen("1") 
        c2[i2] = c2[i2].value == "1" and Gen("0") or Gen("1") 
        c1 = Chrom(c1)
        c2 = Chrom(c2)
        return [c1, c2]
    
    def mutate(self, selected_parents):
        if np.random.randint(1,11,1) < 10 - ( self.mutaion_probability * 10 ): return
        mutated_chroms = self._mutate(selected_parents)
        
        for chrom in mutated_chroms:
            mn = min(self.population, key=lambda x: x.fitness)
            if mn.fitness == 121:
                continue
            mn.genes = chrom.genes
            genetic.fit(genetic.get_population())
            
    
    def crossover(self, selected_parents):
        if np.random.randint(1,11,1) < 10 - ( self.crossover_probabilty * 10 ): return 
        crossed_chromes = []
        if self.crossover_method == 1:
            crossed_chromes = self.single_point_crossover(selected_parents) 
        elif self.crossover_method == 2:
            crossed_chromes = self.double_point_crossover(selected_parents)
        elif self.crossover_method == 3:
            crossed_chromes = self.uniform_crossover(selected_parents)
        mins = np.argsort(self.get_population())
        for chrom in crossed_chromes:
            mn = min(self.population, key=lambda x: x.fitness)
            if mn.fitness == 121:
                continue
            mn.genes = chrom.genes
            self.fit(self.get_population())
            
input_file_directory = "./params.txt"
inputs = []
with open(input_file_directory)as file:
    inputs = file.read().splitlines()
    
genetic = GA(*inputs)
for i in range(genetic.max_iter):
    print("\nPopulation ", i+1)
    genetic.fit(genetic.get_population())
    genetic.print_population()
    parents = genetic.select_parents(2)
    genetic.crossover(parents)
    parents = genetic.select_parents(2)
    genetic.mutate(parents)
    if all([x.fitness == 121 for x in genetic.get_population()]):
        print("\nPopulation ", i+1)
        genetic.print_population()
        print("\nConverged to ", genetic.get_population()[0].genes, "in ", i+1 , "th ", "generation")
        print("Global Maximum of x^2 + 2x function is : ", int("".join(map(str, genetic.get_population()[0].genes)), 2))
        break
        
    



Population  1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1

Population  2
 : [0 0 0 1 1] => 3
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 1 1] => 3
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 0] => 1
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 1 1] => 3
 : [0 0 0 0 1] => 0
 : [0 0 0 0 1] => 0
 : [0 0 0 

 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 0 0 1] => 108
 : [1 1 1 0 1] => 117
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 112
 : [1 1 0 0 1] => 108
 : [1 1 0 0 0] => 109
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 112
 : [1 1 0 1 1] => 111
 : [1 1 0 0 0] => 109
 : [1 1 0 1 1] => 111
 : [1 1 1 0 1] => 117
 : [1 0 0 0 1] => 81
 : [1 1 0 0 0] => 109
 : [1 1 0 0 0] => 109
 : [1 1 0 0 1] => 108
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 112

Population  30
 : [1 1 0 1 1] => 111
 : [1 1 1 0 1] => 117
 : [1 1 0 0 0] => 109
 : [0 1 0 1 0] => 31
 : [0 1 1 1 1] => 39
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 0 0 1] => 108
 : [1 1 1 0 1] => 117
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 112
 : [1 1 0 0 1] => 108
 : [1 1 0 0 0] => 109
 : [0 1 1 1 1] => 39
 : [1 1 0 1 0] => 112
 : [1 1 0 0 1] => 108
 : [1 1 0 1 0] => 112
 : [1 1 0 1 1] => 111
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 112
 : [1 1 0 0 0] => 109
 : [1 1 0 0 0] => 109
 : [1 1 0 0 1] => 108
 : [1 1 1 0 1] => 117
 : [1 1 0 1 0] => 11

 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [0 1 1 1 1] => 39
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [0 1 1 1 1] => 39
 : [1 1 1 1 1] => 120
 : [0 1 1 1 1] => 39
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 1] => 120

Population  58
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 0 0 0] => 109
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 0 0 0] => 109
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 0] => 121
 : [1 1 1 1 1] => 120
 : [1 1 0 0 0] => 1